In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a1283ce548d99146521d471a76bc58a7012ac7390b72260686989fb95f26ddb6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1" # без этой строчки у нас будет возникать постоянное предупреждение с просьбой установить эту переменную в значение 1, что мы заранее и делаем


spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark

In [6]:
products = spark.createDataFrame([
    {"id": 1, "productName": "apples"},
    {"id": 2, "productName": "bananas"},
    {"id": 3, "productName": "cucumners"},
    {"id": 4, "productName": "tomatoes"},
    {"id": 5, "productName": "magazines"},
    {"id": 6, "productName": "cars"},
    {"id": 7, "productName": "bread"}
])

In [7]:
categories = spark.createDataFrame([
    {"id": 1, "categoryName" : "fruit"},
    {"id": 2, "categoryName" : "food"},
    {"id": 3, "categoryName" : "vegetables"}
])

In [8]:
products_categories_pairs = spark.createDataFrame([
    {"id" : 1, "productId" : 1, "categoryId" : 1},
    {"id" : 2, "productId" : 1, "categoryId" : 2},
    {"id" : 3, "productId" : 2, "categoryId" : 1},
    {"id" : 4, "productId" : 2, "categoryId" : 2},
    {"id" : 5, "productId" : 3, "categoryId" : 2},
    {"id" : 6, "productId" : 3, "categoryId" : 3},
    {"id" : 7, "productId" : 4, "categoryId" : 2},
    {"id" : 8, "productId" : 4, "categoryId" : 3},
    {"id" : 9, "productId" : 7, "categoryId" : 2},
])

In [23]:
products\
.join(products_categories_pairs, products.id == products_categories_pairs.productId, "left")\
.join(categories, categories.id == products_categories_pairs.categoryId, "left")\
.orderBy(products.id)\
.select(products.productName, categories.categoryName)\
.show()

+-----------+------------+
|productName|categoryName|
+-----------+------------+
|     apples|       fruit|
|     apples|        food|
|    bananas|       fruit|
|    bananas|        food|
|  cucumners|  vegetables|
|  cucumners|        food|
|   tomatoes|  vegetables|
|   tomatoes|        food|
|  magazines|        NULL|
|       cars|        NULL|
|      bread|        food|
+-----------+------------+

